<a href="https://colab.research.google.com/github/mad7art/work_shop/blob/master/Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up work directory

In [2]:
%mkdir ./sample_data/Lab_5
%ls -a
!pwd

/content/sample_data/Lab_5
./  ../  fish-cat.csv  .ipynb_checkpoints/  test/  train/  val/
/content/sample_data/Lab_5


#**Downloading images**

In [ ]:
#Downloading images into corresponding folders
from csv import reader
import requests
from csv import DictReader

#open file in read mode
with open('./fish-cat.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_dict_reader = DictReader(read_obj)
    # Iterate over each row in the csv using reader object
    file_name_index = 1
    for row in csv_dict_reader:
        #Reading in a row from csv
        url = row['url']
        className = row['class']
        typeName = row['type']
        print(file_name_index)
        try:
          #Downloading file
          r = requests.get(url, timeout=(5, 14), allow_redirects=True)
          if r.ok:
            #Saving file
            open(typeName + '/' + className + '/' + 'Image_' + str(file_name_index) + '.jpg', 'wb').write(r.content)
            file_name_index = file_name_index + 1
        except:
          print('Failed to download:' + url)
    print(file_name_index)

In [4]:
%ls ./train/cat/ | wc -l
%ls ./train/fish/ | wc -l
%ls ./val/cat/ | wc -l
%ls ./val/fish/ | wc -l
%ls ./test/cat/ | wc -l
%ls ./test/fish/ | wc -l

422
393
87
35
91
69


#Removing hidden directories

In [5]:
!rmdir /content/sample_data/Lab_5/train/.ipynb_checkpoints
!rmdir /content/sample_data/Lab_5/test/.ipynb_checkpoints
!rmdir /content/sample_data/Lab_5/val/.ipynb_checkpoints

# Simple network

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms

#Setting up network class
class SimpleNet(nn.Module):
  def __init__(self): 
    super(SimpleNet, self).__init__()
    self.fc1 = nn.Linear(12288, 84)
    self.fc2 = nn.Linear(84, 50)
    self.fc3 = nn.Linear(50,2)

  def forward(self, x):
      x = x.view(-1, 12288)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
  for epoch in range(1, epochs+1):
      training_loss = 0.0
      valid_loss = 0.0
      model.train()
      for batch in train_loader:
          optimizer.zero_grad()
          inputs, targets = batch
          inputs = inputs.to(device)
          targets = targets.to(device)
          output = model(inputs)
          loss = loss_fn(output, targets)
          loss.backward()
          optimizer.step()
          training_loss += loss.data.item() * inputs.size(0)
      training_loss /= len(train_loader.dataset)
      
      model.eval()
      num_correct = 0 
      num_examples = 0
      for batch in val_loader:
          inputs, targets = batch
          inputs = inputs.to(device)
          output = model(inputs)
          targets = targets.to(device)
          loss = loss_fn(output,targets) 
          valid_loss += loss.data.item() * inputs.size(0)
          correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
          num_correct += torch.sum(correct).item()
          num_examples += correct.shape[0]
      valid_loss /= len(val_loader.dataset)

      print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
      valid_loss, num_correct / num_examples))

#Building datasets
#Setting up transformation from image to tensor
transforms = transforms.Compose([
  transforms.Resize((64,64)),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406],
  std=[0.229, 0.224, 0.225] )
  ])

#Train set
train_data_path = "./train"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=transforms)
print(train_data)

#Validation set
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transforms)
print(val_data)

#Test set
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms)
print(test_data)

#Building data loaders
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

simplenet = SimpleNet()

#Initializing optimizer
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, 20)
torch.save(simplenet, "/simplenet")

Dataset ImageFolder
    Number of datapoints: 806
    Root location: ./train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 110
    Root location: ./val/
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 160
    Root location: ./test/
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Epoch: 1, T

# Predictions for fish

In [42]:
from PIL import Image, ImageFile
import torchvision
from torchvision import transforms

labels = ['cat','fish']

transforms = transforms.Compose([
  transforms.Resize((64,64)),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406],
  std=[0.229, 0.224, 0.225] )
  ])

correct = 0
wrong = 0
total = 0

file_name_index = 994
for file_name_index in range(994, 1062): #indexes of fish files in test folder
  img = Image.open("./test/fish/Image_" + str(file_name_index) + ".jpg") 
  img = transforms(img).to('cpu')
  img = torch.unsqueeze(img, 0)

  simplenet.eval()
  prediction = F.softmax(simplenet(img), dim=1)
  prediction = prediction.argmax()
  if labels[prediction] == 'fish':
    correct += 1
  else:
    wrong += 1
  file_name_index += 1
  total += 1

print('Correct: ' + str(correct))
print('Wrong: ' + str(wrong))
print('Total: ' + str(total))
print("Accuracy: %.2f%%" % ((correct/total)*100))

Correct: 52
Wrong: 16
Total: 68
Accuracy: 76.47%


# Predictions for cat

In [43]:
from PIL import Image, ImageFile
import torchvision
from torchvision import transforms

labels = ['cat','fish']

transforms = transforms.Compose([
  transforms.Resize((64,64)),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406],
  std=[0.229, 0.224, 0.225] )
  ])

correct = 0
wrong = 0
total = 0

file_name_index = 994
for file_name_index in range(423, 513): #indexes of fish files in test folder
  img = Image.open("./test/cat/Image_" + str(file_name_index) + ".jpg") 
  img = transforms(img).to('cpu')
  img = torch.unsqueeze(img, 0)

  simplenet.eval()
  prediction = F.softmax(simplenet(img), dim=1)
  prediction = prediction.argmax()
  if labels[prediction] == 'cat':
    correct += 1
  else:
    wrong += 1
  file_name_index += 1
  total += 1

print('Correct: ' + str(correct))
print('Wrong: ' + str(wrong))
print('Total: ' + str(total))
print("Accuracy: %.2f%%" % ((correct/total)*100))

Correct: 68
Wrong: 22
Total: 90
Accuracy: 75.56%
